In [39]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [2]:
text_embedding_model = OpenAIEmbeddings()
db = Chroma(
    collection_name="saup",
    persist_directory="data/output/chromadb",
    embedding_function=text_embedding_model
)

In [67]:
query = "2023년 삼성, 현대, 한전 기타채권의 합"
retriever = db.as_retriever(
    search_type = "similarity_score_threshold",
    search_kwargs={"k": 20, "score_threshold":0.8}
)
docs = retriever.invoke(query)
for doc in docs:
    print(doc.page_content)

{"매입채무및기타채무_0": {"Year": "2023", "Value": {"Samsung": "-", "Hyundai": "-", "Kepco": 9089973, "Sum": 9089973, "Avg": 9089973.0}, "Ancestors": ["부채", "유동부채"]}}
{"충당부채_1": {"Year": "2023", "Value": {"Samsung": 6524876, "Hyundai": 7316877, "Kepco": "-", "Sum": 13841753, "Avg": 6920876.5}, "Ancestors": ["부채", "유동부채"]}}
{"총포괄손익(*3)_0": {"Year": "2023", "Value": {"Samsung": 18837411, "Hyundai": "-", "Kepco": "-", "Sum": 18837411, "Avg": 18837411.0}, "Ancestors": []}}
{"매입채무_0": {"Year": "2023", "Value": {"Samsung": 11319824, "Hyundai": 10952046, "Kepco": "-", "Sum": 22271870, "Avg": 11135935.0}, "Ancestors": ["부채", "유동부채"]}}
{"충당부채_3": {"Year": "2022", "Value": {"Samsung": 5844907, "Hyundai": 8102596, "Kepco": "-", "Sum": 13947503, "Avg": 6973751.5}, "Ancestors": ["부채", "유동부채"]}}
{"자본과부채총계_0": {"Year": "2023", "Value": {"Samsung": 455905980, "Hyundai": "-", "Kepco": 239714965, "Sum": 695620945, "Avg": 347810472.5}, "Ancestors": []}}
{"매출채권_0": {"Year": "2023", "Value": {"Samsung": 36647393, "

In [40]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [62]:
template ="""
    주어진 context 안에서만 대답하고, 절대 임의로 답변을 생성하지 마세요.
    {context}
    단위는 백만원이므로 1원 단위로 바꿔서 대답해 주세요.
    Ancestors가 다른 문서가 있으면 Ancestors 별로 구분해서 답변해 주세요.
    
Question : {question}
"""

In [63]:
prompt = ChatPromptTemplate.from_template(template)

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1,
    max_tokens=500
)

chain = prompt | llm | StrOutputParser()

response = chain.invoke({"context":(format_docs(docs)), "question":query})
print(response)

2023년 삼성, 현대, 한전 기타채권의 합은 다음과 같습니다.

- 삼성: 0 백만원
- 현대: 3,431,169 백만원
- 한전: 0 백만원

합계: 3,431,169 백만원

따라서, 2023년 삼성, 현대, 한전 기타채권의 합은 3,431,169 백만원입니다.
